# Pytorch를 활용한 hand sign 검출

In [4]:
import pandas as pd
import numpy as np
%matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import random
import os
import time
import copy
import glob
from tqdm import tqdm as tqdmd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
from torchvision.io import read_image
import torchvision.models as models
from torchsummary import summary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Using matplotlib backend: Qt5Agg


## Dataset Review

![ASL](./image/01.png)
* ASL(American Sign Language) alphabet은 총 26개가 있지만 dataset에서는 25개 밖에 없다. 
* 그 이유는 Z가 검지로 Z를 그리는 것이기 때문에 사진으로는 표현이 불가능.

## Image dir, path

In [17]:
TRAIN_DIR = "Train/"
TEST_DIR = "Test/"
TRAIN_DIR_LIST = os.listdir(TRAIN_DIR)
TEST_DIR_LIST = os.listdir(TEST_DIR)

In [24]:
os.listdir(TRAIN_DIR+"A/")

['10014_A.jpg',
 '10021_A.jpg',
 '10023_A.jpg',
 '10050_A.jpg',
 '10056_A.jpg',
 '10058_A.jpg',
 '10070_A.jpg',
 '10072_A.jpg',
 '10099_A.jpg',
 '10111_A.jpg',
 '10187_A.jpg',
 '10201_A.jpg',
 '10203_A.jpg',
 '10235_A.jpg',
 '10244_A.jpg',
 '10290_A.jpg',
 '10313_A.jpg',
 '10337_A.jpg',
 '10412_A.jpg',
 '10490_A.jpg',
 '10499_A.jpg',
 '10511_A.jpg',
 '10579_A.jpg',
 '10618_A.jpg',
 '10629_A.jpg',
 '10639_A.jpg',
 '10640_A.jpg',
 '10661_A.jpg',
 '1066_A.jpg',
 '10687_A.jpg',
 '10713_A.jpg',
 '10732_A.jpg',
 '10752_A.jpg',
 '10776_A.jpg',
 '10822_A.jpg',
 '10848_A.jpg',
 '10868_A.jpg',
 '1086_A.jpg',
 '10874_A.jpg',
 '10881_A.jpg',
 '10887_A.jpg',
 '10890_A.jpg',
 '10912_A.jpg',
 '10928_A.jpg',
 '10929_A.jpg',
 '11027_A.jpg',
 '11057_A.jpg',
 '11097_A.jpg',
 '11142_A.jpg',
 '11162_A.jpg',
 '11173_A.jpg',
 '11208_A.jpg',
 '11230_A.jpg',
 '1125_A.jpg',
 '11272_A.jpg',
 '11319_A.jpg',
 '11339_A.jpg',
 '11346_A.jpg',
 '11347_A.jpg',
 '11362_A.jpg',
 '11367_A.jpg',
 '11382_A.jpg',
 '11388_A.j

In [37]:
#Trian Dataset
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TRAIN_DIR_LIST:
    for j in os.listdir(TRAIN_DIR + i +"/"):
        data = {'label' : i, 'dir' : j}
        df = df.append(data, ignore_index=True)
df.to_csv("Train/train.csv")

In [44]:
df = pd.DataFrame(columns = ['label', 'dir'])
for i in TEST_DIR_LIST:
    for j in os.listdir(TEST_DIR + i +"/"):
        data = {'label' : i, 'dir' : j}
        df = df.append(data, ignore_index=True)
df.to_csv("Test/test.csv")

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7172 entries, 0 to 7171
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7172 non-null   object
 1   dir     7172 non-null   object
dtypes: object(2)
memory usage: 112.2+ KB


In [49]:
train_dataset = pd.read_csv("Train/train.csv", index_col = 0)
test_dataset = pd.read_csv("Test/test.csv", index_col = 0)

In [50]:
train_dataset

,label,dir
0,A,10014_A.jpg
1,A,10021_A.jpg
2,A,10023_A.jpg
3,A,10050_A.jpg
4,A,10056_A.jpg
...,...,...
27450,Y,9952_Y.jpg
27451,Y,9973_Y.jpg
27452,Y,9983_Y.jpg
27453,Y,9988_Y.jpg
